#### This lab aims to train and test a classification model using transfer learning capable of classifying the volume and brand of recyclable objects.

#### The pretrained models used in this lab are part of the Keras Applications Reference. The Keras Applications are deep learning models that are made available alongside pre-trained weights. These models can be used for prediction, feature extraction, and fine-tuning.
#### URL link : https://keras.io/api/applications/

#### Within this lab, we will be testing diffrent pretrained models from the Keras Applications. The main goal is to pick up the pretrained model with the heighest performance.

#### Note : As the data collection phase is not completed yet, and as training more than one model can be computationally expensive, this lab will focus on training and testing only one pretrained model to demonstrate the hole process of training and testing a classification model. Once the data collection phase is completed, then we will proceede with the other models.
#### The pretrained model implemented within this lab will be the VGG16 model.You can check the keras Applications for more information about this model and other pretrained models as well.

In [1]:
import os

In [2]:
# Importing the dataset that will be used to train and test the brand and volume classification model
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

MessageError: ignored

In [ ]:
# Just Copying the dataset compressed file from the drive folder to the main folder (/content/)
# The dataset contains images that will be used for training and testing the classification model for all the classes(categories) that we want our model to classify
!cp -r /content/drive/MyDrive/dataset_for_volume_brand_classification.zip .

In [ ]:
# Unzipping the dataset compressed file
!unzip dataset_for_volume_brand_classification.zip

Archive:  dataset_for_volume_brand_classification.zip
   creating: dataset_for_volume_brand_classification/
   creating: dataset_for_volume_brand_classification/BOGA_0.24/
  inflating: dataset_for_volume_brand_classification/BOGA_0.24/boga_0.24i2.jpg  
  inflating: dataset_for_volume_brand_classification/BOGA_0.24/boga_0.24i3.jpg  
  inflating: dataset_for_volume_brand_classification/BOGA_0.24/boga_0.24sm2.jpg  
  inflating: dataset_for_volume_brand_classification/BOGA_0.24/boga_0.24sm3.jpg  
  inflating: dataset_for_volume_brand_classification/BOGA_0.24/boga_0.24sm8.jpg  
  inflating: dataset_for_volume_brand_classification/BOGA_0.24/boga_0.245.jpg  
  inflating: dataset_for_volume_brand_classification/BOGA_0.24/_0_9504.jpg  
  inflating: dataset_for_volume_brand_classification/BOGA_0.24/_0_428.jpg  
  inflating: dataset_for_volume_brand_classification/BOGA_0.24/_0_1334.jpg  
  inflating: dataset_for_volume_brand_classification/BOGA_0.24/_0_4088.jpg  
  inflating: dataset_for_volume_b

In [ ]:
# Remove the  compressed dataset file(no need for the compressed files)
!rm dataset_for_volume_brand_classification.zip

In [ ]:

dataset_directory = 'dataset_for_volume_brand_classification'
dataset_directory

'dataset_for_volume_brand_classification'

In [ ]:
#These are all the classes(categories) we want our model to classify
classes = os.listdir(dataset_directory)
classes

['COCA_0.5', 'SAFIA_1.5', 'BOGA_0.24', 'FOURAT_2.0', 'COCA_0.24']

## Loading Data

In [ ]:
import numpy as np
import pandas as pd
import cv2

In [ ]:
# hyperparameter
IMG_SIZE = 512

In [ ]:
X = []
y = []

for i, category in enumerate(classes):
    for img in os.listdir(os.path.join(dataset_directory, category)):
        img_path = os.path.join(dataset_directory, category, img)
        X.append(cv2.resize(cv2.imread(img_path), (IMG_SIZE , IMG_SIZE)))
        y.append(i)
X = np.asarray(X)

## Scaling Data

In [ ]:
X = X/255.0

## Splitting Data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
SPLIT_SIZE = 0.2

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = SPLIT_SIZE,
    random_state = 42,
    shuffle=True,
    stratify=y
    )

In [ ]:
SPLIT_SIZE = 0.1

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,
    test_size = SPLIT_SIZE,
    random_state = 42,
    shuffle=True,
    stratify=y_train
    )

In [ ]:
y_train = pd.get_dummies(y_train).values

In [ ]:
y_test = pd.get_dummies(y_test).values

In [ ]:
y_val = pd.get_dummies(y_val).values

In [ ]:
y_train.shape

(864, 5)

In [ ]:
y_test.shape

(240, 5)

In [ ]:
y_val.shape

(96, 5)

## Modeling

In [ ]:
import tensorflow as tf
from keras import layers, Model

In [ ]:
model = tf.keras.applications.InceptionV3(
    weights='imagenet',
    input_shape=(512, 512, 3),
    include_top=False,
)
for layer in model.layers:
 layer.trainable = False

last_layer = model.get_layer('mixed7')# Mixed7 is one of the layers in the inception network whose outputs we will use to build the classifier.
last_layer_output = last_layer.output

x = layers.Flatten()(last_layer_output)

x = layers.Dense(64,activation='relu')(x)
x = layers.BatchNormalization()(x)

output = layers.Dense(len(classes), activation='softmax')(x)

model = Model(model.input, output)

NameError: ignored

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

NameError: ignored

In [ ]:
model.summary()

## Training

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# hyperparameters
EPOCHS = 100
BATCH_SIZE = 8

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_accuracy') > 0.99:
            print('\n Validation accuracy = \
            98% so, stopping further training.')
            self.model.stop_training = True


es = EarlyStopping(patience=3,
                   monitor='val_accuracy',
                   restore_best_weights=True)

lr = ReduceLROnPlateau(monitor='val_loss',
                       patience=3,
                       factor=0.5,
                       verbose=1)

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks = [es, lr, myCallback()])

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
fig = plt.figure()

plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()

plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

## Testing

### Saving the model

In [ ]:
model.save('volume_brand_classification_model')

### Convert our model to TFLite
#### TFLite is a lighter version of our model which can be deployed in an embedded system such as a raspberry pi

In [ ]:
model = tf.keras.models.load_model('volume_brand_classification_model')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [ ]:
tflite_model = converter.convert()

In [ ]:
open("volume_brand_classification_model_tflite.tflite", "wb").write(tflite_model)

### Compressing and exporting our model

In [ ]:
!zip -r volume_brand_classification_model.zip volume_brand_classification_model

In [ ]:
!cp volume_brand_classification_model.zip drive/MyDrive

In [ ]:
!zip -r volume_brand_classification_model_tflite.zip volume_brand_classification_model_tflite.tflite

In [ ]:
!cp volume_brand_classification_model_tflite.zip drive/MyDrive